In [ ]:
#統整

In [ ]:
# Prepare data -Preview
#bott
data_bott = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/bottom.csv", inferSchema=True, header=True)

data_bott.show(100,truncate=False)
data_bott.distinct()
data_bott.distinct().count() #13662

#top
data_top = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/top.csv", inferSchema=True, header=True)

data_top.show(100, False)
data_top.distinct()
data_top.distinct().count() #14871

# .....nlp_feature_engineering

In [ ]:
#gnd mapping top&bottom
data_gnd_pre = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/gnd_top_bottom_pairs.csv", inferSchema=True, header=False)
data_gnd = data_gnd_pre.na.drop()
#data_gnd.printSchema()
# data_gnd.show(10,truncate=False)
# data_gnd.distinct().count() #20714

'''rule= 
[ready_data_bott.iid == data_gnd._c1, 
ready_data_top == data_gnd._c0]'''

data_bott=data_bott.withColumnRenamed('iid','bot_id').withColumnRenamed('title', 'bot_Desc').withColumnRenamed('category','bot_Cate')
data_top=data_top.withColumnRenamed('iid','top_id').withColumnRenamed('title', 'top_Desc').withColumnRenamed('category','top_Cate')

#答案集 data_gnd
T_joined_bott = data_gnd.join(data_bott, data_gnd._c1 == data_bott.bot_id, 'inner').select('*')
#joined_bott.show()
#joined_bott.count() #20703
T_joined_bott_top_aswr = T_joined_bott.join(data_top, T_joined_bott._c0 == data_top.top_id, 'inner' ).select('bot_id', 'bot_Desc', 'bot_Cate','top_id','top_Desc','top_Cate')#.show(10)

T_joined_bott_top_aswr = T_joined_bott_top_aswr.withColumn('tag', lit(1))#.show(10) #.count() #20655

#問題集 rand_gnd 
T_joined_bott2 = rand_gnd.join(data_bott, data_gnd._c1 == data_bott.bot_id, 'inner').select('*')
#joined_bott.show()
#joined_bott.count() #20703
T_joined_bott_top_ques = T_joined_bott2.join(data_top, T_joined_bott._c0 == data_top.top_id, 'inner' ).select('bot_id', 'bot_Desc', 'bot_Cate','top_id','top_Desc','top_Cate')#.show(10)

T_joined_bott_top_ques = T_joined_bott_top_ques.withColumn('tag', lit(0))#.show(10) #.count() #20655

#答案資料合併集
pre_dl_dataset_text = T_joined_bott_top_ques.union(T_joined_bott_top_aswr)
pre_dl_dataset_text.show()
pre_dl_dataset_text.count() #41310

#存
pandasDF3 = pre_dl_dataset_text.toPandas()
pandasDF3.to_csv("./pre_dl_dataset_text.csv") 
#pandasDF2.head(10) #.count()

